In [1]:
#%%
# =======================================================================
# CELL 0: ONE-SHOT SETUP WITH KERNEL RESTART (REQUIRED FOR PROTOBUF)
# =======================================================================
print("="*70)
print("COMPLETE SETUP - PROTOBUF FIX + KERNEL RESTART")
print("="*70)

import subprocess
import sys
import os

# Step 1: Uninstall protobuf completely
print("[1/4] Removing old protobuf...")
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "protobuf"], 
               capture_output=True, check=False)
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "protobuf"], 
               capture_output=True, check=False)  # Run twice to ensure clean
print("      ✓ Old protobuf removed")

# Step 2: Install correct version
print("[2/4] Installing protobuf 3.20.3...")
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "protobuf==3.20.3"], 
               capture_output=True, check=True)
print("      ✓ Protobuf 3.20.3 installed")

# Step 3: Install other packages
print("[3/4] Installing packages...")
subprocess.run([
    sys.executable, "-m", "pip", "install", "-q",
    "pandas", "numpy", "scikit-learn",
    "transformers", "torch", "accelerate", "peft",
    "hdbscan", "umap-learn", "datasets", "sentencepiece", "evaluate" , "rouge_score"  , "datasets"
], capture_output=True, check=True)
print("      ✓ Packages installed")

# Step 4: Restart kernel
print("[4/4] Restarting kernel...")
print("\n" + "="*70)
print("⚠️  KERNEL WILL RESTART NOW")
print("⚠️  AFTER RESTART, RUN CELL 1 (NOT THIS CELL AGAIN)")
print("="*70)

# Restart IPython kernel
# os._exit(0)

COMPLETE SETUP - PROTOBUF FIX + KERNEL RESTART
[1/4] Removing old protobuf...
      ✓ Old protobuf removed
[2/4] Installing protobuf 3.20.3...
      ✓ Protobuf 3.20.3 installed
[3/4] Installing packages...
      ✓ Packages installed
[4/4] Restarting kernel...

⚠️  KERNEL WILL RESTART NOW
⚠️  AFTER RESTART, RUN CELL 1 (NOT THIS CELL AGAIN)


In [2]:
#%%
# =======================================================================
# CELL 1: IMPORTS (RUN AFTER KERNEL RESTART)
# =======================================================================
print("="*70)
print("CELL 1: IMPORTING LIBRARIES")
print("="*70)

import json
import pandas as pd
import numpy as np
import os
from collections import defaultdict, Counter
import torch
from transformers import T5EncoderModel, AutoTokenizer, T5TokenizerFast, T5ForConditionalGeneration
from peft import PeftModel
from hdbscan import HDBSCAN
from sklearn.metrics import silhouette_score, davies_bouldin_score
from datasets import Dataset
import warnings
warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully")

# GPU check
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    device = torch.device("cuda")
else:
    print("⚠️ No GPU - using CPU")
    device = torch.device("cpu")

print("\n" + "="*70)
print("✅ CELL 1 COMPLETE - Proceed to Cell 2")
print("="*70)

CELL 1: IMPORTING LIBRARIES


2025-11-09 17:13:45.829923: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762708426.022228      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762708426.074201      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


✅ All libraries imported successfully
✅ GPU: Tesla T4

✅ CELL 1 COMPLETE - Proceed to Cell 2


In [3]:
from pathlib import Path
import torch
import os

# Set the TSV folder
TSV_DATA_DIR = Path("/kaggle/input/syntethic-absa")
assert TSV_DATA_DIR.exists(), f"TSV_DATA_DIR missing: {TSV_DATA_DIR}"

# CL DAPT adapter
DAPT_ADAPTER_DIR = Path("/kaggle/input/cl-training/t5-dapt-contrastive-finetuned")
assert DAPT_ADAPTER_DIR.exists(), f"DAPT_ADAPTER_DIR missing: {DAPT_ADAPTER_DIR}"

OUTPUT_MODEL_DIR = Path("/kaggle/working/flan-t5-dapt-cl-synthetic-full-fft-optimized")
OUTPUT_MODEL_DIR.mkdir(parents=True, exist_ok=True)

BASE_MODEL_NAME = "google/flan-t5-base"

# ============================================================================
# HYPERPARAMETERS (OPTIMIZED - Based on Your Best Config)
# ============================================================================
# Sequence lengths (adjusted for 7-field format)
MAX_INPUT_LENGTH = 384  # Shorter than 512 but safe for your prompts
MAX_TARGET_LENGTH = 96  # Longer than 64 to fit 7 fields

# Batch sizes (match your proven config)
PER_DEVICE_TRAIN_BATCH_SIZE = 8  # Your best: 8
GRADIENT_ACCUMULATION_STEPS = 4  # Your best: 4
PER_DEVICE_EVAL_BATCH_SIZE = 8   # Your best: 8
# Effective batch size = 8 * 4 = 32

# Learning rate and optimizer (match your best)
LEARNING_RATE = 2.5e-4  # Your best: 2.5e-4 (higher than current 1e-4)
OPTIMIZER = "adamw_torch"
WARMUP_RATIO = 0.0  # No warmup (like your best config)
WEIGHT_DECAY = 0.0  # NO WEIGHT DECAY (your best had none)

# Training duration
NUM_TRAIN_EPOCHS = 5  # Same as before

# Generation settings
GEN_NUM_BEAMS = 5  # Keep at 5 (good balance)

# Logging
LOGGING_STEPS = 50

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print(f"TSV data location: {TSV_DATA_DIR}")
print(f"CL DAPT adapter location: {DAPT_ADAPTER_DIR}")
print(f"Output directory: {OUTPUT_MODEL_DIR}")

print("\n" + "="*70)
print("OPTIMIZED HYPERPARAMETERS")
print("="*70)
print(f"Learning Rate:      {LEARNING_RATE} (2.5x higher than before)")
print(f"Weight Decay:       {WEIGHT_DECAY} (DISABLED - matches your best)")
print(f"Batch Size:         {PER_DEVICE_TRAIN_BATCH_SIZE} (2x larger than before)")
print(f"Effective Batch:    {PER_DEVICE_TRAIN_BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")
print(f"Max Input Length:   {MAX_INPUT_LENGTH} (25% shorter)")
print(f"Max Target Length:  {MAX_TARGET_LENGTH} (25% shorter)")
print(f"FP16:               Will be set to FALSE in training args")
print("="*70)

Using device: cuda
TSV data location: /kaggle/input/syntethic-absa
CL DAPT adapter location: /kaggle/input/cl-training/t5-dapt-contrastive-finetuned
Output directory: /kaggle/working/flan-t5-dapt-cl-synthetic-full-fft-optimized

OPTIMIZED HYPERPARAMETERS
Learning Rate:      0.00025 (2.5x higher than before)
Weight Decay:       0.0 (DISABLED - matches your best)
Batch Size:         8 (2x larger than before)
Effective Batch:    32
Max Input Length:   384 (25% shorter)
Max Target Length:  96 (25% shorter)
FP16:               Will be set to FALSE in training args


In [4]:
# ============================================================================
# LOAD AND VERIFY TSV DATA (CORRECTED)
# ============================================================================
import csv
import pandas as pd
from datasets import Dataset
from pathlib import Path
import re

def load_tsv_simple(path):
    """Load TSV with input/target columns, skip header"""
    rows = []
    print(f"Reading from: {path.name}...")
    
    with open(path, "r", encoding="utf-8") as fh:
        reader = csv.reader(fh, delimiter='\t', quoting=csv.QUOTE_MINIMAL)
        header = next(reader, None)  # Skip header
        
        for row in reader:
            if len(row) >= 2:
                rows.append({"input": row[0], "target": row[1]})
    
    print(f"Successfully loaded {len(rows)} rows from {path.name}.")
    return Dataset.from_pandas(pd.DataFrame(rows))

def extract_text_from_prompt(prompt_input):
    """
    Extract the actual text being analyzed from the prompt.
    Looks for the LAST occurrence of 'Text:' before 'Output:'
    """
    # Strategy: Find all "Text: <content>" patterns, take the last one
    all_matches = list(re.finditer(r'Text:\s*(.+?)(?=\s*Output:|$)', prompt_input, re.DOTALL))
    
    if all_matches:
        # Return the LAST match (the actual text to analyze, not examples)
        return all_matches[-1].group(1).strip()
    
    return "[Could not extract text]"

# ============================================================================
# LOAD DATASETS
# ============================================================================
TSV_DIR = Path("/kaggle/input/syntethic-absa")

print("="*60)
print("Loading TSV datasets using Python's csv module...")
print("="*60)

train_ds = load_tsv_simple(TSV_DIR / "train.tsv")
val_ds = load_tsv_simple(TSV_DIR / "val.tsv")
test_ds = load_tsv_simple(TSV_DIR / "test.tsv")

print(f"\n{'='*60}")
print(f"Dataset sizes:")
print(f"  Train: {len(train_ds):,}")
print(f"  Val:   {len(val_ds):,}")
print(f"  Test:  {len(test_ds):,}")
print("="*60)

# ============================================================================
# DISPLAY SAMPLE
# ============================================================================
print("\n--- Train Sample 0 ---")
sample = train_ds[0]

print(f"INPUT:")
print(sample['input'])
print(f"\n\nTARGET:")
print(repr(sample['target']))

# Extract and show actual text
actual_text = extract_text_from_prompt(sample['input'])
print(f"\n\nEXTRACTED ACTUAL TEXT:")
print(actual_text)
print("="*60)

# ============================================================================
# VERIFY MORE SAMPLES
# ============================================================================
print("\n--- Verification: First 5 Samples ---")
for i in range(min(5, len(train_ds))):
    sample = train_ds[i]
    actual_text = extract_text_from_prompt(sample['input'])
    target = sample['target']
    
    print(f"\n{'─'*60}")
    print(f"Sample {i+1}:")
    print(f"Text: {actual_text[:150]}...")
    print(f"Target: {target[:150]}...")
    
    # Count aspects
    aspects = target.split(' ; ')
    print(f"Number of aspects: {len(aspects)}")
    
    # Show first aspect term
    first_aspect = aspects[0].split(' | ')[0] if aspects else ""
    print(f"First aspect: {first_aspect}")

# ============================================================================
# SANITY CHECK: Verify text matches target
# ============================================================================
print("\n" + "="*60)
print("SANITY CHECK: Do extracted texts match targets?")
print("="*60)

mismatch_count = 0
for i in range(min(10, len(train_ds))):
    sample = train_ds[i]
    actual_text = extract_text_from_prompt(sample['input']).lower()
    target = sample['target'].lower()
    
    # Extract aspect from target
    first_aspect = target.split(' | ')[0].split(';')[0].strip()
    
    # Check if aspect term or keywords appear in text
    # For battery_drain, check for "battery" or "drain"
    aspect_keywords = first_aspect.replace('_', ' ').split()
    
    found = any(keyword in actual_text for keyword in aspect_keywords if len(keyword) > 3)
    
    if not found:
        mismatch_count += 1
        print(f"⚠️ Sample {i+1}: Aspect '{first_aspect}' not found in text")
        print(f"   Text: {actual_text[:100]}...")

if mismatch_count == 0:
    print("✅ All samples verified - text and targets match!")
else:
    print(f"⚠️ Found {mismatch_count} potential mismatches")

print("\n" + "="*60)
print("✓ TSV Loading and Verification Complete")
print("="*60)

Loading TSV datasets using Python's csv module...
Reading from: train.tsv...
Successfully loaded 567 rows from train.tsv.
Reading from: val.tsv...
Successfully loaded 122 rows from val.tsv.
Reading from: test.tsv...
Successfully loaded 122 rows from test.tsv.

Dataset sizes:
  Train: 567
  Val:   122
  Test:  122

--- Train Sample 0 ---
INPUT:
Extract aspects and sentiments. Output format: aspect | sentiment | span | rationale | is_implicit | has_slang | has_emoji
Separate multiple aspects with ' ; '.

Example 1:
Text: Great food, slow service.
Output:
food | positive | [1, 1] | User praises food. | FALSE | FALSE | FALSE ; service | negative | [3, 3] | User complains about service speed. | FALSE | FALSE | FALSE

Example 2:
Text: ugh battery draining so fast 😩
Output:
battery | negative | [2, 2] | Complaint about battery life. | FALSE | TRUE | TRUE

Example 3:
Text: This place is beautiful and perfect for dates.
Output:
ambiance | positive | [] | Implicit praise of atmosphere. | TRUE | 

In [5]:
# ============================================================================
# CELL 5: LOAD TOKENIZER AND CL DAPT MODEL (FULL MODEL VERSION)
# ============================================================================
print("="*70)
print("LOADING TOKENIZER AND CL DAPT MODEL")
print("="*70)

from transformers import T5TokenizerFast, T5ForConditionalGeneration

# Load tokenizer
print("\n[1/3] Loading tokenizer...")
tokenizer = T5TokenizerFast.from_pretrained(DAPT_ADAPTER_DIR)
print("✓ Tokenizer loaded")

# Load CL DAPT model directly (not as adapter)
print("\n[2/3] Loading CL DAPT model (full model)...")
model = T5ForConditionalGeneration.from_pretrained(DAPT_ADAPTER_DIR)
print("✓ CL DAPT model loaded")

# Unfreeze all parameters for full fine-tuning
print("\n[3/3] Preparing for fine-tuning...")
for param in model.parameters():
    param.requires_grad = True

# Move to GPU
model = model.to(device)

# Verify trainable params
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("\n" + "="*70)
print("MODEL READY")
print("="*70)
print(f"Total params:     {total_params:,}")
print(f"Trainable params: {trainable_params:,}")
print(f"Trainable %:      {100 * trainable_params / total_params:.2f}%")
print("="*70)

LOADING TOKENIZER AND CL DAPT MODEL

[1/3] Loading tokenizer...
✓ Tokenizer loaded

[2/3] Loading CL DAPT model (full model)...
✓ CL DAPT model loaded

[3/3] Preparing for fine-tuning...

MODEL READY
Total params:     247,577,856
Trainable params: 247,577,856
Trainable %:      100.00%


In [6]:
# ============================================================================
# CELL 6: TOKENIZATION
# ============================================================================
print("="*70)
print("TOKENIZING DATASETS")
print("="*70)

def preprocess_function(batch):
    """
    Tokenize input/target pairs.
    Input: Full prompt with examples + actual text
    Target: aspect | sentiment | span | rationale | is_implicit | has_slang | has_emoji
    """
    # Tokenize inputs (prompts)
    inputs = tokenizer(
        batch["input"], 
        max_length=MAX_INPUT_LENGTH,
        truncation=True,
        padding=False
    )
    
    # Tokenize targets (7-field format)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch["target"], 
            max_length=MAX_TARGET_LENGTH,
            truncation=True,
            padding=False
        )
    
    inputs["labels"] = labels["input_ids"]
    return inputs

print("\nTokenizing train set...")
tokenized_train = train_ds.map(
    preprocess_function, 
    batched=True, 
    remove_columns=train_ds.column_names,
    desc="Tokenizing train"
)

print("Tokenizing validation set...")
tokenized_val = val_ds.map(
    preprocess_function, 
    batched=True, 
    remove_columns=val_ds.column_names,
    desc="Tokenizing val"
)

print("Tokenizing test set...")
tokenized_test = test_ds.map(
    preprocess_function, 
    batched=True, 
    remove_columns=test_ds.column_names,
    desc="Tokenizing test"
)

# Check token lengths
train_input_lens = [len(x) for x in tokenized_train['input_ids']]
train_label_lens = [len(x) for x in tokenized_train['labels']]

print("\n" + "="*70)
print("TOKENIZATION STATISTICS")
print("="*70)
print(f"Input tokens - Min: {min(train_input_lens)}, Max: {max(train_input_lens)}, Avg: {sum(train_input_lens)/len(train_input_lens):.1f}")
print(f"Target tokens - Min: {min(train_label_lens)}, Max: {max(train_label_lens)}, Avg: {sum(train_label_lens)/len(train_label_lens):.1f}")

truncated_inputs = sum(1 for l in train_input_lens if l == MAX_INPUT_LENGTH)
truncated_targets = sum(1 for l in train_label_lens if l == MAX_TARGET_LENGTH)

print(f"\nTruncation: Inputs {truncated_inputs}/{len(train_input_lens)} ({truncated_inputs/len(train_input_lens)*100:.1f}%), Targets {truncated_targets}/{len(train_label_lens)} ({truncated_targets/len(train_label_lens)*100:.1f}%)")
print("="*70)
print("✓ Tokenization Complete")
print("="*70)

TOKENIZING DATASETS

Tokenizing train set...


Tokenizing train:   0%|          | 0/567 [00:00<?, ? examples/s]

Tokenizing validation set...


Tokenizing val:   0%|          | 0/122 [00:00<?, ? examples/s]

Tokenizing test set...


Tokenizing test:   0%|          | 0/122 [00:00<?, ? examples/s]


TOKENIZATION STATISTICS
Input tokens - Min: 292, Max: 384, Avg: 307.5
Target tokens - Min: 25, Max: 96, Avg: 49.7

Truncation: Inputs 28/567 (4.9%), Targets 25/567 (4.4%)
✓ Tokenization Complete


In [7]:
# ============================================================================
# CELL 7: TRAINING SETUP
# ============================================================================
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
import time

print("="*70)
print("TRAINING SETUP")
print("="*70)

# Data collator for dynamic padding
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# Training arguments
# Training arguments (OPTIMIZED)
training_args = Seq2SeqTrainingArguments(
    output_dir=str(OUTPUT_MODEL_DIR),
    
    # Batch sizes (OPTIMIZED)
    per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH_SIZE,  # 8 (was 4)
    per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH_SIZE,    # 8 (was 4)
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,   # 4 (was 8)
    # Effective batch size = 8 * 4 = 32 (same)
    
    # Learning rate and optimizer (OPTIMIZED)
    learning_rate=LEARNING_RATE,  # 2.5e-4 (was 1e-4) ⚡
    optim=OPTIMIZER,
    warmup_ratio=WARMUP_RATIO,    # 0.0 (was 0.1) ⚡
    weight_decay=WEIGHT_DECAY,    # 0.0 (was 0.01) ⚡
    
    # Training duration
    num_train_epochs=NUM_TRAIN_EPOCHS,
    
    # Evaluation and saving
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    save_total_limit=1,
    
    # Generation settings
    predict_with_generate=True,
    generation_max_length=MAX_TARGET_LENGTH,
    generation_num_beams=GEN_NUM_BEAMS,  # 5
    
    # Precision and performance (OPTIMIZED)
    fp16=False,  # Disabled for stability ⚡
    
    # Logging
    logging_strategy="steps",
    logging_steps=LOGGING_STEPS,
    report_to="none",
    
    # Hardware optimization
    dataloader_num_workers=0,
    dataloader_pin_memory=False,
)

# Placeholder compute_metrics (detailed evaluation after training)
def compute_metrics(eval_pred):
    return {}

# Initialize trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("\n" + "="*70)
print("TRAINING CONFIGURATION")
print("="*70)
print(f"Effective batch size: {PER_DEVICE_TRAIN_BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")
print(f"Training samples: {len(tokenized_train)}")
print(f"Steps per epoch: {len(tokenized_train) // (PER_DEVICE_TRAIN_BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS)}")
print(f"Total training steps: ~{(len(tokenized_train) // (PER_DEVICE_TRAIN_BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS)) * NUM_TRAIN_EPOCHS}")
print(f"Estimated time: ~{((len(tokenized_train) // (PER_DEVICE_TRAIN_BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS)) * NUM_TRAIN_EPOCHS * 1.5 / 60):.0f} minutes")
print("="*70)
print("✓ Trainer Ready")
print("="*70)

TRAINING SETUP

TRAINING CONFIGURATION
Effective batch size: 32
Training samples: 567
Steps per epoch: 17
Total training steps: ~85
Estimated time: ~2 minutes
✓ Trainer Ready


In [8]:
# ============================================================================
# CELL 8: TRAINING
# ============================================================================
print("="*70)
print("STARTING FINE-TUNING")
print("="*70)
print(f"Model: FLAN-T5-Base + DAPT (Span-Masked) + CL")
print(f"Task: ABSA (7-field format)")
print(f"Dataset: {len(tokenized_train)} train, {len(tokenized_val)} val")
print("="*70)

start_time = time.time()
model.train()

# Start training
train_result = trainer.train()

end_time = time.time()
training_minutes = (end_time - start_time) / 60

print("\n" + "="*70)
print("TRAINING COMPLETE")
print("="*70)
print(f"Training time: {training_minutes:.1f} minutes ({training_minutes/60:.2f} hours)")
print(f"Final train loss: {train_result.metrics.get('train_loss', 'N/A'):.4f}")
print("="*70)

# Save metrics
trainer.log_metrics("train", train_result.metrics)
trainer.save_metrics("train", train_result.metrics)

# Save best model
print(f"\nSaving best model to {OUTPUT_MODEL_DIR}...")
trainer.save_model(str(OUTPUT_MODEL_DIR))
tokenizer.save_pretrained(str(OUTPUT_MODEL_DIR))
print("✓ Model and tokenizer saved")

STARTING FINE-TUNING
Model: FLAN-T5-Base + DAPT (Span-Masked) + CL
Task: ABSA (7-field format)
Dataset: 567 train, 122 val


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,No log,0.987630
2,No log,0.726520
3,No log,0.629680
4,No log,0.595783
5,No log,0.584395


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].



TRAINING COMPLETE
Training time: 7.7 minutes (0.13 hours)
Final train loss: 1.0871
***** train metrics *****
  epoch                    =        5.0
  total_flos               =  1303462GF
  train_loss               =     1.0871
  train_runtime            = 0:07:41.99
  train_samples_per_second =      6.136
  train_steps_per_second   =      0.097

Saving best model to /kaggle/working/flan-t5-dapt-cl-synthetic-full-fft-optimized...
✓ Model and tokenizer saved


In [9]:
# ============================================================================
# CELL 9: GENERATE PREDICTIONS ON TEST SET
# ============================================================================
import torch
from tqdm.notebook import tqdm
import json

print("="*70)
print("GENERATING PREDICTIONS")
print("="*70)

# Load best model
print(f"Loading best model from {OUTPUT_MODEL_DIR}...")
model = T5ForConditionalGeneration.from_pretrained(OUTPUT_MODEL_DIR).to(device)
tokenizer = T5TokenizerFast.from_pretrained(OUTPUT_MODEL_DIR)
model.eval()
print("✓ Model loaded\n")

def generate_predictions(dataset, tokenizer, model, device, batch_size=PER_DEVICE_EVAL_BATCH_SIZE):
    """Generate predictions with progress bar"""
    all_preds = []
    
    for i in tqdm(range(0, len(dataset), batch_size), desc="Generating"):
        batch = dataset[i:i + batch_size]
        input_texts = batch['input']
        gold_targets = batch['target']
        
        # Tokenize
        inputs = tokenizer(
            input_texts, 
            return_tensors="pt", 
            padding=True, 
            truncation=True, 
            max_length=MAX_INPUT_LENGTH
        ).to(device)
        
        # Generate
        with torch.no_grad():
            generated_ids = model.generate(
                **inputs,
                num_beams=GEN_NUM_BEAMS,
                max_length=MAX_TARGET_LENGTH,
                early_stopping=True
            )
        
        # Decode
        generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        
        # Store results
        for j in range(len(input_texts)):
            all_preds.append({
                'input': input_texts[j],
                'gold_target': gold_targets[j],
                'gen_output': generated_texts[j]
            })
    
    return all_preds

# Generate predictions
print(f"Generating predictions on {len(test_ds)} test samples...\n")
test_predictions = generate_predictions(test_ds, tokenizer, model, device)

# Save predictions
PRED_FILE = OUTPUT_MODEL_DIR / "test_predictions.jsonl"
with open(PRED_FILE, 'w', encoding='utf-8') as f:
    for pred in test_predictions:
        f.write(json.dumps(pred, ensure_ascii=False) + '\n')

print(f"\n✓ Predictions saved to {PRED_FILE}")
print("="*70)

GENERATING PREDICTIONS
Loading best model from /kaggle/working/flan-t5-dapt-cl-synthetic-full-fft-optimized...
✓ Model loaded

Generating predictions on 122 test samples...



Generating:   0%|          | 0/16 [00:00<?, ?it/s]


✓ Predictions saved to /kaggle/working/flan-t5-dapt-cl-synthetic-full-fft-optimized/test_predictions.jsonl


In [10]:
# ============================================================================
# CELL 10: EVALUATE PREDICTIONS (7-FIELD FORMAT)
# ============================================================================
import re
from collections import Counter

print("="*70)
print("EVALUATION - 7-FIELD ABSA FORMAT")
print("="*70)

# ============================================================================
# PARSING FUNCTIONS
# ============================================================================

def normalize_aspect(s):
    """Normalize aspect term"""
    if not s: return ''
    return re.sub(r'\s+', ' ', s.strip().lower())

def normalize_sentiment(s):
    """Normalize sentiment to positive/negative/neutral"""
    if not s: return ''
    s = s.strip().lower()
    if s.startswith('pos'): return 'positive'
    if s.startswith('neg'): return 'negative'
    if s.startswith('neu'): return 'neutral'
    return s

def parse_7field_output(text):
    """
    Parse 7-field format: aspect | sentiment | span | rationale | is_implicit | has_slang | has_emoji
    Returns: set of (aspect, sentiment) tuples for matching
    """
    pairs = set()
    if not text or not text.strip():
        return pairs
    
    # Split by ' ; ' separator
    entries = re.split(r'\s*;\s*', text.strip())
    
    for entry in entries:
        # Split by ' | ' to get fields
        fields = [f.strip() for f in entry.split('|')]
        
        if len(fields) >= 2:
            aspect = normalize_aspect(fields[0])
            sentiment = normalize_sentiment(fields[1])
            
            if aspect and sentiment in ['positive', 'negative', 'neutral']:
                pairs.add((aspect, sentiment))
    
    return pairs

# ============================================================================
# METRICS COMPUTATION
# ============================================================================

def compute_prf(tp, fp, fn):
    """Compute Precision, Recall, F1"""
    p = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    r = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f = 2 * p * r / (p + r) if (p + r) > 0 else 0.0
    return p, r, f

# Counters
TP = FP = FN = 0  # Aspect + Sentiment (tuple matching)
TP_aspect = FP_aspect = FN_aspect = 0  # Aspect-only
exact_match = 0
total_samples = len(test_predictions)

# Sentiment accuracy on detected aspects
sentiment_correct = 0
detected_aspects = 0

# Per-aspect stats
aspect_counts_gold = Counter()
aspect_counts_pred = Counter()
aspect_tp = Counter()

print(f"\nEvaluating {total_samples} test samples...\n")

# ============================================================================
# EVALUATION LOOP
# ============================================================================

for item in test_predictions:
    gold_text = item['gold_target']
    pred_text = item['gen_output']
    
    # Parse both
    gold_pairs = parse_7field_output(gold_text)
    pred_pairs = parse_7field_output(pred_text)
    
    # Tuple-level (aspect + sentiment)
    TP += len(gold_pairs & pred_pairs)
    FP += len(pred_pairs - gold_pairs)
    FN += len(gold_pairs - pred_pairs)
    
    # Exact match
    if gold_pairs == pred_pairs:
        exact_match += 1
    
    # Aspect-only (ignore sentiment)
    gold_aspects = set(a for a, _ in gold_pairs)
    pred_aspects = set(a for a, _ in pred_pairs)
    
    TP_aspect += len(gold_aspects & pred_aspects)
    FP_aspect += len(pred_aspects - gold_aspects)
    FN_aspect += len(gold_aspects - pred_aspects)
    
    # Per-aspect counts
    for asp in gold_aspects:
        aspect_counts_gold[asp] += 1
    for asp in pred_aspects:
        aspect_counts_pred[asp] += 1
    for asp in (gold_aspects & pred_aspects):
        aspect_tp[asp] += 1
    
    # Sentiment accuracy on detected aspects
    for asp in (gold_aspects & pred_aspects):
        detected_aspects += 1
        gold_sents = set(s for a, s in gold_pairs if a == asp)
        pred_sents = set(s for a, s in pred_pairs if a == asp)
        if gold_sents & pred_sents:
            sentiment_correct += 1

# ============================================================================
# COMPUTE METRICS
# ============================================================================

tuple_p, tuple_r, tuple_f1 = compute_prf(TP, FP, FN)
aspect_p, aspect_r, aspect_f1 = compute_prf(TP_aspect, FP_aspect, FN_aspect)
exact_acc = exact_match / total_samples if total_samples > 0 else 0.0
sent_acc = sentiment_correct / detected_aspects if detected_aspects > 0 else 0.0

# ============================================================================
# DISPLAY RESULTS
# ============================================================================

print("="*70)
print("PRIMARY METRICS")
print("="*70)
print(f"\n1. Aspect + Sentiment (Tuple Matching):")
print(f"   TP: {TP}, FP: {FP}, FN: {FN}")
print(f"   Precision: {tuple_p:.4f}")
print(f"   Recall:    {tuple_r:.4f}")
print(f"   F1 Score:  {tuple_f1:.4f}")

print(f"\n2. Aspect-Only (Ignore Sentiment):")
print(f"   TP: {TP_aspect}, FP: {FP_aspect}, FN: {FN_aspect}")
print(f"   Precision: {aspect_p:.4f}")
print(f"   Recall:    {aspect_r:.4f}")
print(f"   F1 Score:  {aspect_f1:.4f}")

print(f"\n3. Exact Match Accuracy:")
print(f"   {exact_match}/{total_samples} = {exact_acc:.4f} ({exact_acc*100:.2f}%)")

print(f"\n4. Sentiment Accuracy (on detected aspects):")
print(f"   {sentiment_correct}/{detected_aspects} = {sent_acc:.4f} ({sent_acc*100:.2f}%)")

# ============================================================================
# TOP ASPECTS
# ============================================================================

print("\n" + "="*70)
print("TOP 10 ASPECTS")
print("="*70)

top_aspects = aspect_counts_gold.most_common(10)
print(f"\n{'Aspect':<30} {'Gold':>6} {'Pred':>6} {'TP':>6} {'F1':>8}")
print("-"*70)

for asp, gold_count in top_aspects:
    pred_count = aspect_counts_pred.get(asp, 0)
    tp_count = aspect_tp.get(asp, 0)
    
    p = tp_count / pred_count if pred_count > 0 else 0.0
    r = tp_count / gold_count if gold_count > 0 else 0.0
    f1 = 2 * p * r / (p + r) if (p + r) > 0 else 0.0
    
    print(f"{asp:<30} {gold_count:>6} {pred_count:>6} {tp_count:>6} {f1:>8.4f}")

print("\n" + "="*70)
print("✓ EVALUATION COMPLETE")
print("="*70)

EVALUATION - 7-FIELD ABSA FORMAT

Evaluating 122 test samples...

PRIMARY METRICS

1. Aspect + Sentiment (Tuple Matching):
   TP: 54, FP: 71, FN: 82
   Precision: 0.4320
   Recall:    0.3971
   F1 Score:  0.4138

2. Aspect-Only (Ignore Sentiment):
   TP: 70, FP: 55, FN: 65
   Precision: 0.5600
   Recall:    0.5185
   F1 Score:  0.5385

3. Exact Match Accuracy:
   48/122 = 0.3934 (39.34%)

4. Sentiment Accuracy (on detected aspects):
   54/70 = 0.7714 (77.14%)

TOP 10 ASPECTS

Aspect                           Gold   Pred     TP       F1
----------------------------------------------------------------------
sound_quality                       5      2      2   0.5714
taste                               5      3      3   0.7500
battery_life                        5      4      3   0.6667
atmosphere                          4      3      3   0.8571
vibe                                4      2      2   0.6667
food_quality                        4      1      1   0.4000
screen_quality       

In [11]:
# ============================================================================
# CELL 11: SHOW PREDICTION EXAMPLES
# ============================================================================
import re

def extract_actual_text(prompt):
    """Extract the actual text from prompt"""
    matches = list(re.finditer(r'Text:\s*(.+?)(?=\s*Output:|$)', prompt, re.DOTALL))
    if matches:
        return matches[-1].group(1).strip()
    return "[Could not extract]"

print("="*70)
print("PREDICTION EXAMPLES (First 10)")
print("="*70)

for i in range(min(10, len(test_predictions))):
    item = test_predictions[i]
    actual_text = extract_actual_text(item['input'])
    
    print(f"\n{'─'*70}")
    print(f"Example {i+1}")
    print(f"{'─'*70}")
    print(f"TEXT:\n{actual_text[:200]}...")
    print(f"\nGOLD:\n{item['gold_target']}")
    print(f"\nPREDICTED:\n{item['gen_output']}")
    
    # Quick match check
    gold_pairs = parse_7field_output(item['gold_target'])
    pred_pairs = parse_7field_output(item['gen_output'])
    match = "✅ EXACT MATCH" if gold_pairs == pred_pairs else f"❌ Partial ({len(gold_pairs & pred_pairs)}/{len(gold_pairs)} matched)"
    print(f"\n{match}")

print("\n" + "="*70)

PREDICTION EXAMPLES (First 10)

──────────────────────────────────────────────────────────────────────
Example 1
──────────────────────────────────────────────────────────────────────
TEXT:
tried the new cafe downtown, food taste was meh but portions were huge tho, worth it for the deal...

GOLD:
food_taste | neutral | [4, 4] | Explicit neutral: 'food taste was meh' indicates average flavor without strong praise or criticism. | FALSE | FALSE | FALSE

PREDICTED:
food_taste | negative | [0, 1] | Explicit negative: 'food taste was meh' bland. | FALSE | FALSE | FALSE ; portions | negative | [0, 1] | Explicit negative: 'portions were huge' bland. | FALSE | FALSE | FALSE

❌ Partial (0/1 matched)

──────────────────────────────────────────────────────────────────────
Example 2
──────────────────────────────────────────────────────────────────────
TEXT:
traffic on the highway is brutal rn, took me 90 mins for what should be 30 wtf 🚗...

GOLD:
traffic | negative | [0, 1] | Explicit negative: 't

In [12]:
!pip install bert_score
# ============================================================================
# CELL 12: GENERATION QUALITY METRICS
# ============================================================================
print("="*70)
print("GENERATION QUALITY METRICS")
print("="*70)

# ============================================================================
# 1. ROUGE SCORES
# ============================================================================
print("\n[1/4] Computing ROUGE scores...")
import evaluate

try:
    rouge = evaluate.load('rouge')
    
    gold_strings = [item['gold_target'] for item in test_predictions]
    pred_strings = [item['gen_output'] for item in test_predictions]
    
    rouge_results = rouge.compute(
        predictions=pred_strings,
        references=gold_strings,
        use_stemmer=True
    )
    
    print("\n✅ ROUGE Scores:")
    print(f"   ROUGE-1: {rouge_results['rouge1']:.4f}")
    print(f"   ROUGE-2: {rouge_results['rouge2']:.4f}")
    print(f"   ROUGE-L: {rouge_results['rougeL']:.4f}")
    
except Exception as e:
    print(f"   ❌ ROUGE failed: {e}")

# ============================================================================
# 2. BLEU SCORE
# ============================================================================
print("\n[2/4] Computing BLEU score...")
try:
    from sacrebleu import corpus_bleu
    
    # BLEU expects list of references (each ref is a list)
    references = [[gold] for gold in gold_strings]
    
    bleu_result = corpus_bleu(pred_strings, references)
    
    print(f"\n✅ BLEU Score: {bleu_result.score:.4f}")
    print(f"   (BLEU-1: {bleu_result.precisions[0]:.2f}, "
          f"BLEU-2: {bleu_result.precisions[1]:.2f}, "
          f"BLEU-3: {bleu_result.precisions[2]:.2f}, "
          f"BLEU-4: {bleu_result.precisions[3]:.2f})")
    
except ImportError:
    print("   ⚠️ Installing sacrebleu...")
    import subprocess
    subprocess.run(["pip", "install", "-q", "sacrebleu"], check=True)
    from sacrebleu import corpus_bleu
    references = [[gold] for gold in gold_strings]
    bleu_result = corpus_bleu(pred_strings, references)
    print(f"\n✅ BLEU Score: {bleu_result.score:.4f}")
except Exception as e:
    print(f"   ❌ BLEU failed: {e}")

# ============================================================================
# 3. METEOR SCORE
# ============================================================================
print("\n[3/4] Computing METEOR score...")
try:
    meteor = evaluate.load('meteor')
    
    meteor_result = meteor.compute(
        predictions=pred_strings,
        references=gold_strings
    )
    
    print(f"\n✅ METEOR Score: {meteor_result['meteor']:.4f}")
    
except Exception as e:
    print(f"   ❌ METEOR failed: {e}")

# ============================================================================
# 4. BERTScore
# ============================================================================
print("\n[4/4] Computing BERTScore (this may take a minute)...")
try:
    bertscore = evaluate.load('bertscore')
    
    bert_results = bertscore.compute(
        predictions=pred_strings,
        references=gold_strings,
        lang='en',
        model_type='microsoft/deberta-base-mnli',  # Faster than roberta-large
        batch_size=16
    )
    
    # Average scores
    avg_precision = sum(bert_results['precision']) / len(bert_results['precision'])
    avg_recall = sum(bert_results['recall']) / len(bert_results['recall'])
    avg_f1 = sum(bert_results['f1']) / len(bert_results['f1'])
    
    print(f"\n✅ BERTScore:")
    print(f"   Precision: {avg_precision:.4f}")
    print(f"   Recall:    {avg_recall:.4f}")
    print(f"   F1:        {avg_f1:.4f}")
    
except Exception as e:
    print(f"   ❌ BERTScore failed: {e}")
    print("   (This is optional - task-specific metrics are more important)")

print("\n" + "="*70)
print("✓ GENERATION METRICS COMPLETE")
print("="*70)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.7 MB/s eta 0:00:00
GENERATION QUALITY METRICS

[1/4] Computing ROUGE scores...



✅ ROUGE Scores:
   ROUGE-1: 0.5583
   ROUGE-2: 0.3612
   ROUGE-L: 0.5459

[2/4] Computing BLEU score...
   ⚠️ Installing sacrebleu...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.3 MB/s eta 0:00:00

✅ BLEU Score: 83.1077

[3/4] Computing METEOR score...


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...



✅ METEOR Score: 0.6401

[4/4] Computing BERTScore (this may take a minute)...


...

config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/557M [00:00<?, ?B/s]


✅ BERTScore:
   Precision: 0.7776
   Recall:    0.7614
   F1:        0.7686

✓ GENERATION METRICS COMPLETE


In [13]:
# ============================================================================
# CELL 13: SEMANTIC HALLUCINATION & COVERAGE ANALYSIS
# ============================================================================
import re
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

print("="*70)
print("SEMANTIC HALLUCINATION & COVERAGE ANALYSIS")
print("="*70)

# ============================================================================
# LOAD SEMANTIC SIMILARITY MODEL
# ============================================================================
print("\n[0/4] Loading semantic similarity model...")
try:
    semantic_model = SentenceTransformer('all-MiniLM-L6-v2')  # Fast, lightweight
    print("✓ Semantic model loaded")
except:
    print("⚠️ Installing sentence-transformers...")
    import subprocess
    subprocess.run(["pip", "install", "-q", "sentence-transformers"], check=True)
    semantic_model = SentenceTransformer('all-MiniLM-L6-v2')
    print("✓ Semantic model loaded")

SEMANTIC_THRESHOLD = 0.65  # Cosine similarity threshold (0.65-0.75 is good)

def is_semantically_similar(aspect1, aspect2, threshold=SEMANTIC_THRESHOLD):
    """Check if two aspects are semantically similar"""
    # Handle exact match first
    if aspect1.lower() == aspect2.lower():
        return True, 1.0
    
    # Normalize underscores to spaces
    asp1 = aspect1.replace('_', ' ')
    asp2 = aspect2.replace('_', ' ')
    
    # Compute embeddings
    emb1 = semantic_model.encode([asp1])
    emb2 = semantic_model.encode([asp2])
    
    # Compute cosine similarity
    sim = cosine_similarity(emb1, emb2)[0][0]
    
    return sim >= threshold, sim

def extract_actual_text(prompt):
    """Extract actual text from prompt"""
    matches = list(re.finditer(r'Text:\s*(.+?)(?=\s*Output:|$)', prompt, re.DOTALL))
    if matches:
        return matches[-1].group(1).strip()
    return ""

# ============================================================================
# 1. SEMANTIC HALLUCINATION DETECTION
# ============================================================================
print("\n[1/4] Analyzing hallucinations with semantic similarity...")

explicit_correct = 0
implicit_semantically_valid = 0
true_hallucinations = 0
total_generated_aspects = 0

# Store examples
hallucination_examples = []
semantic_match_examples = []

for idx, item in enumerate(test_predictions):
    actual_text = extract_actual_text(item['input']).lower()
    gold_output = item['gold_target'].lower()
    gen_output = item['gen_output'].lower()
    
    # Parse gold aspects
    gold_aspects = set()
    for entry in re.split(r'\s*;\s*', gold_output):
        fields = entry.split('|')
        if fields:
            gold_aspects.add(fields[0].strip())
    
    # Check generated aspects
    for entry in re.split(r'\s*;\s*', gen_output):
        fields = entry.split('|')
        if not fields or not fields[0].strip():
            continue
        
        aspect = fields[0].strip()
        total_generated_aspects += 1
        
        # Category 1: Explicit (appears in text)
        if aspect.replace('_', ' ') in actual_text:
            explicit_correct += 1
        else:
            # Category 2: Check semantic similarity with gold aspects
            is_semantic_match = False
            best_sim = 0.0
            best_match = None
            
            for gold_asp in gold_aspects:
                is_similar, sim = is_semantically_similar(aspect, gold_asp)
                if is_similar and sim > best_sim:
                    is_semantic_match = True
                    best_sim = sim
                    best_match = gold_asp
            
            if is_semantic_match:
                implicit_semantically_valid += 1
                if len(semantic_match_examples) < 5:
                    semantic_match_examples.append({
                        'generated': aspect,
                        'gold': best_match,
                        'similarity': best_sim,
                        'text': actual_text[:100]
                    })
            else:
                # Category 3: True hallucination (not in text, not similar to gold)
                true_hallucinations += 1
                if len(hallucination_examples) < 5:
                    hallucination_examples.append({
                        'generated': aspect,
                        'gold_aspects': list(gold_aspects),
                        'text': actual_text[:100]
                    })

# Compute rates
semantic_hallucination_rate = true_hallucinations / total_generated_aspects if total_generated_aspects > 0 else 0
strict_hallucination_rate = (true_hallucinations + implicit_semantically_valid) / total_generated_aspects if total_generated_aspects > 0 else 0

print(f"\n✅ Semantic Hallucination Analysis:")
print(f"   Total generated aspects:           {total_generated_aspects}")
print(f"   Explicit (in text):                {explicit_correct} ({explicit_correct/total_generated_aspects*100:.1f}%)")
print(f"   Implicit but semantically valid:   {implicit_semantically_valid} ({implicit_semantically_valid/total_generated_aspects*100:.1f}%)")
print(f"   True hallucinations:               {true_hallucinations} ({semantic_hallucination_rate*100:.1f}%)")
print(f"\n   📊 Semantic Hallucination Rate:    {semantic_hallucination_rate:.4f} ({semantic_hallucination_rate*100:.1f}%)")
print(f"   📊 Strict Hallucination Rate:      {strict_hallucination_rate:.4f} ({strict_hallucination_rate*100:.1f}%)")

# Show examples
if semantic_match_examples:
    print(f"\n   Examples of semantic matches (threshold={SEMANTIC_THRESHOLD}):")
    for i, ex in enumerate(semantic_match_examples, 1):
        print(f"   {i}. Generated: '{ex['generated']}' ≈ Gold: '{ex['gold']}' (sim={ex['similarity']:.3f})")

if hallucination_examples:
    print(f"\n   ⚠️ Examples of true hallucinations:")
    for i, ex in enumerate(hallucination_examples, 1):
        print(f"   {i}. Generated: '{ex['generated']}' | Gold: {ex['gold_aspects']}")
        print(f"      Text: {ex['text']}...")

# ============================================================================
# 2. SEMANTIC COVERAGE ANALYSIS
# ============================================================================
print("\n[2/4] Analyzing coverage with semantic matching...")

total_gold_aspects = 0
exact_matches = 0
semantic_matches = 0
missed_aspects = 0

coverage_examples = []

for item in test_predictions:
    gold_pairs = parse_7field_output(item['gold_target'])
    pred_pairs = parse_7field_output(item['gen_output'])
    
    gold_aspects = set(a for a, _ in gold_pairs)
    pred_aspects = set(a for a, _ in pred_pairs)
    
    for gold_asp in gold_aspects:
        total_gold_aspects += 1
        
        # Check for exact match
        if gold_asp in pred_aspects:
            exact_matches += 1
        else:
            # Check for semantic match
            found_semantic = False
            for pred_asp in pred_aspects:
                is_similar, sim = is_semantically_similar(gold_asp, pred_asp)
                if is_similar:
                    semantic_matches += 1
                    found_semantic = True
                    break
            
            if not found_semantic:
                missed_aspects += 1
                if len(coverage_examples) < 5:
                    coverage_examples.append({
                        'missed': gold_asp,
                        'predicted': list(pred_aspects),
                        'text': extract_actual_text(item['input'])[:100]
                    })

# Compute coverage rates
exact_coverage = exact_matches / total_gold_aspects if total_gold_aspects > 0 else 0
semantic_coverage = (exact_matches + semantic_matches) / total_gold_aspects if total_gold_aspects > 0 else 0

print(f"\n✅ Semantic Coverage Analysis:")
print(f"   Total gold aspects:       {total_gold_aspects}")
print(f"   Exact matches:            {exact_matches} ({exact_coverage*100:.1f}%)")
print(f"   Semantic matches:         {semantic_matches} ({semantic_matches/total_gold_aspects*100:.1f}%)")
print(f"   Missed aspects:           {missed_aspects} ({missed_aspects/total_gold_aspects*100:.1f}%)")
print(f"\n   📊 Exact Coverage:        {exact_coverage:.4f} ({exact_coverage*100:.1f}%)")
print(f"   📊 Semantic Coverage:     {semantic_coverage:.4f} ({semantic_coverage*100:.1f}%)")

if coverage_examples:
    print(f"\n   ⚠️ Examples of missed aspects:")
    for i, ex in enumerate(coverage_examples, 1):
        print(f"   {i}. Missed: '{ex['missed']}' | Predicted: {ex['predicted']}")
        print(f"      Text: {ex['text']}...")

# ============================================================================
# 3. FORMAT ADHERENCE (same as before)
# ============================================================================
print("\n[3/4] Checking format adherence...")

valid_format = 0
malformed = 0
empty_outputs = 0

for item in test_predictions:
    gen_output = item['gen_output'].strip()
    
    if not gen_output:
        empty_outputs += 1
        continue
    
    entries = re.split(r'\s*;\s*', gen_output)
    all_valid = True
    
    for entry in entries:
        fields = entry.split('|')
        if len(fields) != 7:
            all_valid = False
            break
        
        if len(fields) >= 2:
            sent = fields[1].strip().lower()
            if sent not in ['positive', 'negative', 'neutral']:
                all_valid = False
                break
    
    if all_valid:
        valid_format += 1
    else:
        malformed += 1

format_adherence = valid_format / len(test_predictions) if test_predictions else 0

print(f"\n✅ Format Adherence:")
print(f"   Valid 7-field format: {valid_format}/{len(test_predictions)} ({format_adherence*100:.1f}%)")
print(f"   Malformed outputs:    {malformed} ({malformed/len(test_predictions)*100:.1f}%)")
print(f"   Empty outputs:        {empty_outputs} ({empty_outputs/len(test_predictions)*100:.1f}%)")

# ============================================================================
# 4. QUALITY SCORE
# ============================================================================
print("\n[4/4] Computing overall quality score...")

# Weighted quality score
quality_score = (
    0.4 * semantic_coverage +           # 40% weight on finding aspects
    0.3 * (1 - semantic_hallucination_rate) +  # 30% weight on avoiding hallucinations
    0.2 * format_adherence +            # 20% weight on format
    0.1 * (explicit_correct / total_generated_aspects if total_generated_aspects > 0 else 0)  # 10% weight on groundedness
)

print(f"\n✅ Overall Quality Score: {quality_score:.4f} ({quality_score*100:.1f}%)")
print(f"   Interpretation:")
if quality_score >= 0.80:
    print(f"   🟢 Excellent (≥80%) - Production ready")
elif quality_score >= 0.65:
    print(f"   🟡 Good (65-80%) - Acceptable for research")
elif quality_score >= 0.50:
    print(f"   🟠 Fair (50-65%) - Needs improvement")
else:
    print(f"   🔴 Poor (<50%) - Requires significant work")

print("\n" + "="*70)
print("✓ SEMANTIC ANALYSIS COMPLETE")
print("="*70)

# Store semantic metrics
semantic_metrics = {
    'semantic_hallucination_rate': semantic_hallucination_rate,
    'strict_hallucination_rate': strict_hallucination_rate,
    'semantic_coverage': semantic_coverage,
    'exact_coverage': exact_coverage,
    'quality_score': quality_score
}

SEMANTIC HALLUCINATION & COVERAGE ANALYSIS

[0/4] Loading semantic similarity model...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✓ Semantic model loaded

[1/4] Analyzing hallucinations with semantic similarity...

✅ Semantic Hallucination Analysis:
   Total generated aspects:           140
   Explicit (in text):                108 (77.1%)
   Implicit but semantically valid:   8 (5.7%)
   True hallucinations:               24 (17.1%)

   📊 Semantic Hallucination Rate:    0.1714 (17.1%)
   📊 Strict Hallucination Rate:      0.2286 (22.9%)

   Examples of semantic matches (threshold=0.65):
   1. Generated: 'policy_policy' ≈ Gold: 'healthcare_policy' (sim=0.708)
   2. Generated: 'support_reply' ≈ Gold: 'support' (sim=0.753)
   3. Generated: 'battery_life' ≈ Gold: 'battery_life' (sim=1.000)
   4. Generated: 'battery_life' ≈ Gold: 'battery_life' (sim=1.000)
   5. Generated: 'food_taste' ≈ Gold: 'food_taste' (sim=1.000)

   ⚠️ Examples of true hallucinations:
   1. Generated: 'app_quality' | Gold: ['software']
      Text: oh great, another update that breaks the app, thanks for nothing devs 🙄...
   2. Generated: 'app_qu

In [14]:
# ============================================================================
# CELL 14: FINAL RESULTS SUMMARY (WITH SEMANTIC METRICS)
# ============================================================================
print("\n" + "="*70)
print("COMPREHENSIVE EVALUATION SUMMARY")
print("="*70)
print(f"\nModel: FLAN-T5-Base + DAPT + CL")
print(f"Dataset: {len(test_predictions)} test samples")
print(f"Training time: {training_minutes:.1f} minutes")
print(f"Final train loss: {train_result.metrics.get('train_loss', 0):.4f}")

print("\n" + "="*70)
print("TASK-SPECIFIC METRICS (Primary)")
print("="*70)
print(f"\n1. Aspect + Sentiment F1:      {tuple_f1:.4f}")
print(f"2. Aspect-Only F1:              {aspect_f1:.4f}")
print(f"3. Exact Match Accuracy:        {exact_acc:.4f}")
print(f"4. Sentiment Accuracy:          {sent_acc:.4f}")

print("\n" + "="*70)
print("GENERATION QUALITY METRICS")
print("="*70)
try:
    print(f"\n5. ROUGE-L:                     {rouge_results['rougeL']:.4f}")
    print(f"6. BLEU:                        {bleu_result.score:.4f}")
    print(f"7. METEOR:                      {meteor_result['meteor']:.4f}")
    print(f"8. BERTScore F1:                {avg_f1:.4f}")
except:
    print("\n(Some generation metrics unavailable)")

print("\n" + "="*70)
print("RELIABILITY METRICS (Semantic-Aware)")
print("="*70)
print(f"\n9. Semantic Hallucination Rate: {semantic_hallucination_rate:.4f} ({semantic_hallucination_rate*100:.1f}%)")
print(f"   (Strict Hallucination Rate:  {strict_hallucination_rate:.4f} ({strict_hallucination_rate*100:.1f}%))")
print(f"\n10. Semantic Coverage:          {semantic_coverage:.4f} ({semantic_coverage*100:.1f}%)")
print(f"    (Exact Coverage:             {exact_coverage:.4f} ({exact_coverage*100:.1f}%))")
print(f"\n11. Format Adherence:           {format_adherence:.4f} ({format_adherence*100:.1f}%)")
print(f"\n12. Overall Quality Score:      {quality_score:.4f} ({quality_score*100:.1f}%)")

# Quality interpretation
print(f"\n    Quality Assessment: ", end="")
if quality_score >= 0.80:
    print(f"🟢 Excellent (Production Ready)")
elif quality_score >= 0.65:
    print(f"🟡 Good (Research Acceptable)")
elif quality_score >= 0.50:
    print(f"🟠 Fair (Needs Improvement)")
else:
    print(f"🔴 Poor (Requires Major Work)")

print("\n" + "="*70)
print("ASPECT DETECTION BREAKDOWN")
print("="*70)
print(f"\nTotal generated aspects: {total_generated_aspects}")
print(f"  ├─ Explicit (in text):              {explicit_correct} ({explicit_correct/total_generated_aspects*100:.1f}%)")
print(f"  ├─ Implicit (semantic match):       {implicit_semantically_valid} ({implicit_semantically_valid/total_generated_aspects*100:.1f}%)")
print(f"  └─ True hallucinations:             {true_hallucinations} ({semantic_hallucination_rate*100:.1f}%)")

print("\n" + "="*70)
print("✅ EVALUATION COMPLETE")
print("="*70)

# ============================================================================
# SAVE COMPREHENSIVE SUMMARY
# ============================================================================
summary = {
    'metadata': {
        'model': 'FLAN-T5-Base + DAPT (Span-Masked)',
        'dataset_size': len(test_predictions),
        'training_time_minutes': float(training_minutes),
        'train_loss': float(train_result.metrics.get('train_loss', 0)),
        'evaluation_date': '2025-11-09',
        'user': 'saheenus-pg'
    },
    
    'task_specific_metrics': {
        'aspect_sentiment_f1': float(tuple_f1),
        'aspect_only_f1': float(aspect_f1),
        'exact_match_accuracy': float(exact_acc),
        'sentiment_accuracy': float(sent_acc)
    },
    
    'generation_quality_metrics': {},
    
    'reliability_metrics': {
        'semantic_hallucination_rate': float(semantic_hallucination_rate),
        'strict_hallucination_rate': float(strict_hallucination_rate),
        'semantic_coverage': float(semantic_coverage),
        'exact_coverage': float(exact_coverage),
        'format_adherence': float(format_adherence),
        'overall_quality_score': float(quality_score)
    },
    
    'aspect_breakdown': {
        'total_generated': int(total_generated_aspects),
        'explicit_in_text': int(explicit_correct),
        'implicit_semantic_match': int(implicit_semantically_valid),
        'true_hallucinations': int(true_hallucinations)
    },
    
    'interpretation': {
        'quality_level': 'Excellent' if quality_score >= 0.80 else 
                        'Good' if quality_score >= 0.65 else 
                        'Fair' if quality_score >= 0.50 else 'Poor',
        'production_ready': bool(quality_score >= 0.80),
        'research_acceptable': bool(quality_score >= 0.65)
    }
}

# Add generation metrics if available
try:
    summary['generation_quality_metrics'] = {
        'rouge_l': float(rouge_results['rougeL']),
        'bleu': float(bleu_result.score),
        'meteor': float(meteor_result['meteor']),
        'bertscore_f1': float(avg_f1)
    }
except:
    summary['generation_quality_metrics'] = {
        'note': 'Some metrics unavailable'
    }

# Save to JSON
summary_file = OUTPUT_MODEL_DIR / 'eval_summary_semantic.json'
with open(summary_file, 'w') as f:
    json.dump(summary, f, indent=2)

print(f"\n✓ Comprehensive summary saved to {summary_file}")

# ============================================================================
# SAVE HUMAN-READABLE REPORT
# ============================================================================
report_file = OUTPUT_MODEL_DIR / 'evaluation_report.txt'
with open(report_file, 'w') as f:
    f.write("="*70 + "\n")
    f.write("ABSA MODEL EVALUATION REPORT\n")
    f.write("="*70 + "\n")
    f.write(f"\nModel: FLAN-T5-Base + DAPT (Span-Masked)\n")
    f.write(f"Evaluated: 2025-11-09\n")
    f.write(f"By: saheenus-pg\n")
    f.write(f"Test Samples: {len(test_predictions)}\n")
    f.write(f"Training Time: {training_minutes:.1f} minutes\n")
    f.write(f"Final Train Loss: {train_result.metrics.get('train_loss', 0):.4f}\n")
    
    f.write("\n" + "="*70 + "\n")
    f.write("KEY FINDINGS\n")
    f.write("="*70 + "\n")
    f.write(f"\n✓ Overall Quality: {quality_score:.1%} ({summary['interpretation']['quality_level']})\n")
    f.write(f"✓ Aspect Detection F1: {aspect_f1:.1%}\n")
    f.write(f"✓ Semantic Coverage: {semantic_coverage:.1%}\n")
    f.write(f"✓ Hallucination Rate: {semantic_hallucination_rate:.1%}\n")
    f.write(f"✓ Format Adherence: {format_adherence:.1%}\n")
    
    f.write("\n" + "="*70 + "\n")
    f.write("DETAILED METRICS\n")
    f.write("="*70 + "\n")
    f.write(f"\nTask-Specific:\n")
    f.write(f"  - Aspect + Sentiment F1: {tuple_f1:.4f}\n")
    f.write(f"  - Aspect-Only F1: {aspect_f1:.4f}\n")
    f.write(f"  - Exact Match Accuracy: {exact_acc:.4f}\n")
    f.write(f"  - Sentiment Accuracy: {sent_acc:.4f}\n")
    
    f.write(f"\nReliability:\n")
    f.write(f"  - Semantic Hallucination Rate: {semantic_hallucination_rate:.4f}\n")
    f.write(f"  - Semantic Coverage: {semantic_coverage:.4f}\n")
    f.write(f"  - Format Adherence: {format_adherence:.4f}\n")
    
    f.write(f"\nAspect Breakdown:\n")
    f.write(f"  - Explicit (in text): {explicit_correct}/{total_generated_aspects} ({explicit_correct/total_generated_aspects*100:.1f}%)\n")
    f.write(f"  - Implicit (semantic): {implicit_semantically_valid}/{total_generated_aspects} ({implicit_semantically_valid/total_generated_aspects*100:.1f}%)\n")
    f.write(f"  - Hallucinations: {true_hallucinations}/{total_generated_aspects} ({semantic_hallucination_rate*100:.1f}%)\n")
    
    f.write("\n" + "="*70 + "\n")
    f.write("RECOMMENDATIONS\n")
    f.write("="*70 + "\n")
    
    if quality_score >= 0.80:
        f.write("\n✅ Model is production-ready for ABSA tasks.\n")
    elif quality_score >= 0.65:
        f.write("\n✅ Model is acceptable for research purposes.\n")
        f.write("💡 Suggestions for improvement:\n")
        if semantic_coverage < 0.75:
            f.write("   - Increase training data to improve coverage\n")
        if semantic_hallucination_rate > 0.10:
            f.write("   - Train longer or add regularization to reduce hallucinations\n")
    else:
        f.write("\n⚠️ Model needs significant improvement.\n")
        f.write("💡 Critical actions:\n")
        f.write("   - Collect more diverse training data\n")
        f.write("   - Train for more epochs (10-15)\n")
        f.write("   - Consider simplifying output format\n")
    
    f.write("\n" + "="*70 + "\n")

print(f"✓ Human-readable report saved to {report_file}")

print("\n" + "="*70)
print("📊 ALL EVALUATION ARTIFACTS SAVED")
print("="*70)
print(f"  1. {summary_file.name}")
print(f"  2. {report_file.name}")
print(f"  3. test_predictions.jsonl")
print("="*70)


COMPREHENSIVE EVALUATION SUMMARY

Model: FLAN-T5-Base + DAPT + CL
Dataset: 122 test samples
Training time: 7.7 minutes
Final train loss: 1.0871

TASK-SPECIFIC METRICS (Primary)

1. Aspect + Sentiment F1:      0.4138
2. Aspect-Only F1:              0.5385
3. Exact Match Accuracy:        0.3934
4. Sentiment Accuracy:          0.7714

GENERATION QUALITY METRICS

5. ROUGE-L:                     0.5459
6. BLEU:                        83.1077
7. METEOR:                      0.6401
8. BERTScore F1:                0.7686

RELIABILITY METRICS (Semantic-Aware)

9. Semantic Hallucination Rate: 0.1714 (17.1%)
   (Strict Hallucination Rate:  0.2286 (22.9%))

10. Semantic Coverage:          0.6074 (60.7%)
    (Exact Coverage:             0.5185 (51.9%))

11. Format Adherence:           0.9918 (99.2%)

12. Overall Quality Score:      0.7670 (76.7%)

    Quality Assessment: 🟡 Good (Research Acceptable)

ASPECT DETECTION BREAKDOWN

Total generated aspects: 140
  ├─ Explicit (in text):              108